In [1]:
import glob
import pandas as pd
import time
import docx
import PyPDF2
import cv2
import pytesseract
from datetime import date
import re
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

In [2]:
DF = pd.DataFrame(columns=["fileName", "text"])
st_time = time.time()
paths = glob.glob(r"C:\Users\QuadAdmin\Downloads\aclImdb\all\**\*.txt",recursive = True)
print(f"time took: {(time.time() - st_time):.2f} seconds")

time took: 0.48 seconds


In [3]:
# paths

In [4]:
DF["fileName"] = paths

In [5]:
DF

,fileName,text
0,C:\Users\QuadAdmin\Downloads\aclImdb\all\test\...,NaN
1,C:\Users\QuadAdmin\Downloads\aclImdb\all\test\...,NaN
2,C:\Users\QuadAdmin\Downloads\aclImdb\all\test\...,NaN
3,C:\Users\QuadAdmin\Downloads\aclImdb\all\test\...,NaN
4,C:\Users\QuadAdmin\Downloads\aclImdb\all\test\...,NaN
...,...,...
49995,C:\Users\QuadAdmin\Downloads\aclImdb\all\train...,NaN
49996,C:\Users\QuadAdmin\Downloads\aclImdb\all\train...,NaN
49997,C:\Users\QuadAdmin\Downloads\aclImdb\all\train...,NaN
49998,C:\Users\QuadAdmin\Downloads\aclImdb\all\train...,NaN


In [6]:
import findspark
findspark.init()

In [7]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [8]:
conf = SparkConf()
conf.set("spark.network.timeout", "1000000")
conf.set("spark.executor.heartbeatInterval", "1000000")
conf.set("spark.speculation","false")
conf.set("spark.app.name","blaher")
spark = SparkSession.builder.config(conf = conf).master("local[2]").enableHiveSupport().getOrCreate() #("local[*]") uses all the cpu cores#

KeyboardInterrupt: 

In [ ]:
spark

In [ ]:
sc = spark.sparkContext

In [ ]:
# ls_ran = list(range(1,41))
# len(ls_ran)

In [ ]:
# ls_rdd = sc.parallelize(ls_ran)
# ls_rdd.take(5)

In [ ]:
# type(ls_rdd)

In [ ]:
# ls_rdd.getNumPartitions()

In [ ]:
def readfiles(path):
    with open(path,"r",encoding="utf-8") as file:
        text = file.read()
    return text

In [ ]:
DF["fileName"][0]

In [ ]:
# readfiles(DF["fileName"][0])
re.sub("\n"," ",readfiles(DF["fileName"][0]))

In [ ]:
paths_rdd = sc.parallelize(paths)

In [ ]:
# text_rdd = paths_rdd.map(readfiles)

In [ ]:
start_time = time.time()
DF["text"] = DF["fileName"].apply(lambda path: readfiles(path))
print(f"time took: {(time.time() - start_time):.2f} seconds")

In [ ]:
DF

In [ ]:
DF["text"] = DF["text"].apply(lambda text: re.sub("\n"," ",text))
# re.sub('<[^>]*>', '', text)
DF["text"] = DF["text"].apply(lambda text: re.sub('<[^>]*>', '',text))

In [ ]:
DF

In [ ]:
from pyspark.sql.types import *

In [ ]:
from pyspark.sql import *

In [ ]:
import pyspark.sql.functions as F

In [ ]:
from parse_pii.piiparser import *

In [ ]:
DF["text"][0]

In [ ]:
classify(DF["text"][0])

In [ ]:
sparkDF = spark.createDataFrame(DF)

In [ ]:
sparkDF.show(5)

In [ ]:
sparkDF.rdd.getNumPartitions()

In [ ]:
classify_udf = F.udf(classify, StringType())

In [ ]:
# start_time = time.time()
# sparkDF.select("fileName", classify_udf("text").alias("has_pii")).show(5)
# print(f"time took: {(time.time() - start_time):.2f} seconds")

In [ ]:
# fDF = sparkDF.select("fileName", classify_udf("text").alias("has_pii"))

In [ ]:
# start_time = time.time()
# fDF.coalesce(2).write.mode("overwrite").format("csv").option("header","true").save("output")
# print(f"time took: {(time.time() - start_time):.2f} seconds")

In [ ]:
# fDF.printSchema()

In [ ]:
import spacy
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import SpacyNlpEngine
from presidio_analyzer.recognizer_registry import RecognizerRegistry

In [ ]:
def entity_counter(entity_list):
    freq = {}
    entity_list = sorted(entity_list)
    for items in entity_list: 
        freq[items] = entity_list.count(items) 
 
    counter = ""
    for key, value in freq.items(): 
        if key == entity_list[-1]:
            counter += f"{key}:{value}"
        else:
            counter += f"{key}:{value}"+", "
    return counter

In [ ]:
analyzer = AnalyzerEngine(nlp_engine=nlp)

In [ ]:
def pii_classify(stext):
    text = stext
    response = analyzer.analyze(correlation_id=0, text = text,language="en",entities=[], score_threshold=0.5)
    if len(response) == 0:
        return "no"
    else:
        return "yes"

In [ ]:
(DF["text"][1])

In [ ]:
pii_classify(DF["text"][1])

In [ ]:
pii_apply_udf = F.udf(pii_classify, StringType())

In [ ]:
sparkDF.select("fileName", classify_udf("text").alias("has_pii")).show(4)

In [ ]:
fDF = sparkDF.select("fileName", classify_udf("text").alias("has_pii"))

In [ ]:
# fDF.write.mode("overwrite").saveAsTable("op_table")

In [ ]:
# writing only path and content
sparkDF.write.mode("overwrite").saveAsTable("raw_table_texts_2")

In [ ]:
# start_time = time.time()
# sparkDF.coalesce(2).write.mode("overwrite").format("csv").option("header","true").save("output")
# print(f"time took: {(time.time() - start_time):.2f} seconds")

In [ ]:
# getText_Docx("C:\\Users\\QuadAdmin\\Downloads\\files\\docs\\5.doc")

In [ ]:
# with open("C:\\Users\\QuadAdmin\\Downloads\\files\\docs\\5.doc","rb") as doc:
#     print(doc.read())

In [ ]:
# import textract
# text = textract.process(r"C:\Users\QuadAdmin\Downloads\files\docs\5.doc")

In [ ]:
# import olefile

In [ ]:
# filename = r"C:\\Users\\QuadAdmin\\Downloads\\files\\docs\\5.doc"

In [ ]:
# assert olefile.isOleFile(filename)

In [ ]:
# with olefile.OleFileIO("5.doc") as ole:
#     stream = ole.openstream('WordDocument')
#     data = stream.read()